# Project 2
## Team B, CA Beaches


In [107]:
# set environment
from datetime import date
import pandas as pd
import json
import requests

In [108]:
from sqlalchemy import create_engine, insert
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
#from config import password

In [109]:
# define our current grade data url
grade_url = "https://admin.beachreportcard.org/api/locations"

In [110]:
# read JSON data from web
gr = requests.get(grade_url)

#print(gr.json())
# create list of column names
key_list = ["title", "name1", "geo", "address", "city", "state", "zip", "county"]



In [111]:
# create empty beach dict
beach_data = {}


# loop through all the beaches we scraped
for beach in gr.json():
    # we only want cali data
    if beach["_source"]["state"] == "CA":
        
        # add dictionary entry, key = id, value = {}
        beach_data[beach["_source"]["id"]] = {}

        # loop through the list of keys we want to pull from this dataset
        for key in key_list:
               
            # check for data associated with this key
            if key in beach["_source"]:
                    
                if key == "geo":
                    # separate coordinates
                    beach_data[beach["_source"]["id"]]["latitude"] = beach["_source"][key][0]
                    beach_data[beach["_source"]["id"]]["longitude"] = beach["_source"][key][1]
                else:
                    # we have data in this field, add it to our array
                    beach_data[beach["_source"]["id"]][key] = beach["_source"][key]
            else:
                # no data found for this column name --
                # set to empty string
                beach_data[beach["_source"]["id"]][key] = ""
                
#print(beach_data)
        


In [112]:
# define our historical data URL
hist_url = "https://admin.beachreportcard.org/api/grades"

In [114]:
# read JSON data from web
hist = requests.get(hist_url)

#print(hist.json())

In [121]:
# create empty beach dict
hist_data = {"id": [], "title": [], "name1":[], "latitude": [], "longitude": [], "address": [], "city": [], "state": [], "zip": [], "county": [], "grade_updated": [], "dry_grade":[], "wet_grade": []}

#print(hist_data)

# loop through the array of historical entries,
# populate missing data with data from current grade data
for row in hist.json():
    if row["_source"]["LocationId"] in beach_data:
  
        hist_data["id"].append(row["_source"]["LocationId"])
        hist_data["title"].append(beach_data[row["_source"]["LocationId"]]["title"])
        hist_data["name1"].append(beach_data[row["_source"]["LocationId"]]["name1"])
        hist_data["latitude"].append(beach_data[row["_source"]["LocationId"]]["latitude"])
        hist_data["longitude"].append(beach_data[row["_source"]["LocationId"]]["longitude"])
        hist_data["address"].append(beach_data[row["_source"]["LocationId"]]["address"])
        hist_data["city"].append(beach_data[row["_source"]["LocationId"]]["city"])
        hist_data["state"].append(beach_data[row["_source"]["LocationId"]]["state"])
        hist_data["zip"].append(beach_data[row["_source"]["LocationId"]]["zip"])
        hist_data["county"].append(beach_data[row["_source"]["LocationId"]]["county"])
        hist_data["grade_updated"].append(row["_source"]["GradeDate"])
        hist_data["dry_grade"].append(row["_source"]["DryGrade"])
        hist_data["wet_grade"].append(row["_source"]["WetGrade"])


#print(hist_data)
#print(len(hist_data))

In [122]:
# dump data into dataframe
hist_df = pd.DataFrame(hist_data)

In [124]:
# display dataframe
hist_df.head()

,id,title,name1,latitude,longitude,address,city,state,zip,county,grade_updated,dry_grade,wet_grade
0,41,Dockweiler Beach- opposite Hyperion Treatment ...,Dockweiler State Beach,33.918933000000000,-118.431592000000000,None,Playa Del Rey,CA,90293,Los Angeles,2008-04-29,A,ns
1,44,Manhattan Beach Pier drain,Manhattan Beach,33.883814000000000,-118.413234000000000,,Manhattan Beach,CA,90266,Los Angeles -- Redondo,2008-04-29,A+,ns
2,37,Dockweiler State Beach at Ballona Creek mouth,Dockweiler State Beach,33.961154700000000,-118.456217900000000,None,Playa Del Rey,CA,90293,Los Angeles,2008-04-29,A,ns
3,38,Dockweiler State Beach at Culver Blvd. drain,Dockweiler State Beach,33.956461000000000,-118.451844000000000,,Playa Del Rey,CA,90293,Los Angeles -- LA City,2008-04-29,A+,ns
4,42,Dockweiler State Beach at Grand Ave. drain,Dockweiler State Beach,33.915612000000000,-118.429727000000000,None,Manhattan Beach,CA,90266,Los Angeles,2008-04-29,A+,ns


In [125]:
 "active",  "annual_summer_dry", "annual_year_wet", "annual_winter_dry", "annual_year", "grade_created", "alerts"
hist_df["active"] = ""
hist_df["annual_summer_dry"] = ""
hist_df["annual_year_wet"] = ""
hist_df["annual_winter_dry"] = ""
hist_df["annual_year"] = ""
hist_df["grade_created"] = ""
hist_df["alerts"] = "False"

In [126]:
hist_df.head()

,id,title,name1,latitude,longitude,address,city,state,zip,county,grade_updated,dry_grade,wet_grade,active,annual_summer_dry,annual_year_wet,annual_winter_dry,annual_year,grade_created,alerts
0,41,Dockweiler Beach- opposite Hyperion Treatment ...,Dockweiler State Beach,33.918933000000000,-118.431592000000000,None,Playa Del Rey,CA,90293,Los Angeles,2008-04-29,A,ns,,,,,,,False
1,44,Manhattan Beach Pier drain,Manhattan Beach,33.883814000000000,-118.413234000000000,,Manhattan Beach,CA,90266,Los Angeles -- Redondo,2008-04-29,A+,ns,,,,,,,False
2,37,Dockweiler State Beach at Ballona Creek mouth,Dockweiler State Beach,33.961154700000000,-118.456217900000000,None,Playa Del Rey,CA,90293,Los Angeles,2008-04-29,A,ns,,,,,,,False
3,38,Dockweiler State Beach at Culver Blvd. drain,Dockweiler State Beach,33.956461000000000,-118.451844000000000,,Playa Del Rey,CA,90293,Los Angeles -- LA City,2008-04-29,A+,ns,,,,,,,False
4,42,Dockweiler State Beach at Grand Ave. drain,Dockweiler State Beach,33.915612000000000,-118.429727000000000,None,Manhattan Beach,CA,90266,Los Angeles,2008-04-29,A+,ns,,,,,,,False


In [127]:
# write dataframe to a CSV file
hist_df.to_csv("data/hist_info.csv")